<a href="https://colab.research.google.com/github/chrismarkella/Kaggle-access-from-Google-Colab/blob/master/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Metrics to measure how good are the predictions:
- Accuracy
- Precision
- Recall/Sensitivity
- F1 score
- R2 score
- ...

In [1]:
import sklearn.metrics as met

for score_method in filter(lambda s: s.find('score') > 0, dir(met)):
    print(score_method)

_scorer
accuracy_score
adjusted_mutual_info_score
adjusted_rand_score
average_precision_score
balanced_accuracy_score
brier_score_loss
calinski_harabasz_score
calinski_harabaz_score
cohen_kappa_score
completeness_score
consensus_score
davies_bouldin_score
dcg_score
explained_variance_score
f1_score
fbeta_score
fowlkes_mallows_score
get_scorer
homogeneity_score
jaccard_score
jaccard_similarity_score
label_ranking_average_precision_score
make_scorer
mutual_info_score
ndcg_score
normalized_mutual_info_score
precision_recall_fscore_support
precision_score
r2_score
recall_score
roc_auc_score
silhouette_score
v_measure_score


###Setup

In [0]:
import numpy as np

from sklearn.datasets import fetch_openml

In [0]:
mnist = fetch_openml(name='mnist_784', version=1)

In [4]:
X, y = mnist['data'], mnist['target']
y = y.astype(np.uint8)

TRAIN_SIZE = 60000

x_train, x_test, y_train, y_test = X[:TRAIN_SIZE], X[TRAIN_SIZE:], y[:TRAIN_SIZE], y[TRAIN_SIZE:]
for _set in (x_train, x_test, y_train, y_test):
    print(_set.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


###Binary Classifier
- We classify the digits as `five` or `not five(0,1,2,3,4,6,7,8,9)`

In [0]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

###Our `NeverFiveClassifier` always predict `not five`

In [0]:
from sklearn.base import BaseEstimator

class NeverFiveClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass

    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_five_clf = NeverFiveClassifier()

In [0]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(estimator=never_five_clf, X=x_train, y=y_train_5, cv=3)

###Confusion matrix

- `rows` are the `actual` values
- `columns` are the `predicted` values
- since this is a `binary classifier`, we have `only two classes`(`five` and `not five`)
- `first` row and column are the `negative class`(`non five`)
- `second` row and column are the `positive class`(`five`)

In [8]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_true=y_train_5, y_pred=y_train_pred)

array([[54579,     0],
       [ 5421,     0]])

###TP, TN, FP, FN
- True Positive(`TP`): predicted positive and the actual value is positive
- True Negative(`TN`): predicted negative and the actual value is negative
- False Positive(`FP`): predicted positive but the actual value is negative
- False Negative(`FN`): predicted negative and the actual value is positive
---
- TN = 54579
- TP = 0
- FP = 0
- FN = 5421

###Our accuracy is close to `91 percent`, but in reality we just say `not five`.
This is actually the right predictions most of the time, since only about 10 percent of the digits are `fives`.

However we `never predicted any fives`.
`precision` and `recall` is a better `metric` here. They are both show `zero` scores.

In [9]:
from sklearn.metrics import (accuracy_score, precision_score,
                             recall_score, f1_score) 

metric_functions = (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,)

for func in metric_functions:
    score = func(y_true=y_train_5, y_pred=y_train_pred)
    print(f'{func.__name__:20}: {score}')

accuracy_score      : 0.90965
precision_score     : 0.0
recall_score        : 0.0
f1_score            : 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
